## ProbIoU

cls : roundabout|| Recall: 0.9771986962726394 || Precison: 0.41594454072790293|| AP: 0.8564599037811349

F1:0.8869376773191997 P:0.867601246105919 R:0.907166123039767

mAP is : 0.8564599037811349

## GWD

cls : roundabout|| Recall: 0.9690553738037008 || Precison: 0.32179556517036234|| AP: 0.8220850479377181

F1:0.8511415316072127 P:0.8141263940520446 R:0.8916938103487835

mAP is : 0.8220850479377181

### Compilations

In [ ]:
#%cd /workdir/msc/RotationDetection/libs/utils/cython_utils
#!rm *.so
#!rm *.c
#!rm *.cpp
#!python setup.py build_ext --inplace

In [ ]:
#%cd /workdir/msc/RotationDetection/libs/utils/
#!rm *.so
#!rm *.c
#!rm *.cpp
#!python setup.py build_ext --inplace

### Dataset

#### HRSC2016

In [ ]:
## change path in files to attend each dataset
#%cd /workdir/msc/RotationDetection/dataloader/dataset/HRSC2016
#!python make_test_xml.py

In [ ]:
#%cd /workdir/msc/RotationDetection/dataloader/dataset/  
#!python convert_data_to_tfrecord.py --VOC_dir='/datasets/dataset/HRSC2016/HRSC2016/HRSC2016'  \
#                                   --xml_dir='/datasets/dataset/HRSC2016/HRSC2016/HRSC2016/Train/xmls' \
#                                   --image_dir='/datasets/dataset/HRSC2016/HRSC2016/HRSC2016/Train/AllImages' \
#                                   --save_name='train'  \
#                                   --img_format='.bmp'  \
#                                   --save_dir='/datasets/dataset/HRSC2016/HRSC2016' \
#                                   --dataset='HRSC2016'     \

#### DOTA

In [ ]:
%cd /workdir/msc/RotationDetection/dataloader/dataset/DOTA
!python data_crop.py

In [ ]:
%cd /workdir/msc/RotationDetection/dataloader/dataset/  
!python convert_data_to_tfrecord.py --VOC_dir='/datasets/dataset/DOTA/crop/trainval' \
                                   --xml_dir='labeltxt' \
                                   --image_dir='images' \
                                   --save_name='train'  \
                                   --img_format='.png'  \
                                   --dataset='DOTA'

### COCO_VOC

In [5]:
# change path in files to attend each dataset
%cd /workdir/msc/RotationDetection/dataloader/dataset/COCO_VOC
!python txt2xml.py

/workdir/msc/RotationDetection/dataloader/dataset/COCO_VOC
True
100%|█████████████████████████████████████| 85241/85241 [40:41<00:00, 34.91it/s]


In [6]:
%cd /workdir/msc/RotationDetection/dataloader/dataset/  
!python convert_data_to_tfrecord.py --VOC_dir='/datasets/dataset/COCO_VOC/train' \
                                   --xml_dir='xmls' \
                                   --image_dir='images' \
                                   --save_name='train'  \
                                   --img_format='.jpg'  \
                                   --dataset='COCO_VOC'

/workdir/msc/RotationDetection/dataloader/dataset
2021-05-22 01:42:03.498087: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.11.0
/workdir/msc/RotationDetection

100%|█████████████████████████████████████| 85240/85240 [46:51<00:00, 30.32it/s]

Conversion is complete!


## Train

In [ ]:
%cd /workdir/msc/RotationDetection/tools/gwd
!python train.py

/workdir/msc/RotationDetection/tools/gwd
2021-05-22 02:49:01.344093: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.11.0
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

/workdir/msc/RotationDetection







tfrecord path is --> /datasets/tfrecord/COCO_VOC_train*











**********
Using ProbIoU l3
**********

**********
Using ProbIoU l3
**********



model restore from pretrained mode, path is : /workdir/msc/RotationDetection/dataloader/pretrained_weights/resnet50_v1d.ckpt
var_in_graph:  resnet50_v1d/C1/conv0/weights:0
var_in_ckpt:  resnet50_v1d/C1/conv0/weights
______________________________________________

## Test

In [ ]:
!rm -r /workdir/msc/RotationDetection/tools/gwd/test_dota/RetinaNet_DOTA_1x_20201217/dota_res

In [3]:
%cd /workdir/msc/RotationDetection/tools/gwd
!python test_dota.py --test_dir='/datasets/dataset/DOTA/test/images'  \
                     --gpus=0,1

/workdir/msc/RotationDetection/tools/gwd
2021-05-21 13:02:38.519306: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.11.0
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





/workdir/msc/RotationDetection
************************************************************************************
Already tested imgs: []
************************************************************************************
  0%|                                                   | 0/937 [00:00<?, ?it/s]process:0, start:0, end:469
process:1, start:469, end:937












******************************
 CHECKPOINT PATH: /workdir/msc/Rotation

In [ ]:
!ls /datasets/dataset/DOTA/val/labels/labelTxt-v1.0

In [ ]:
!cat /datasets/dataset/DOTA/val/labels/labelTxt-v1.0/P0003.txt

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread('/datasets/dataset/DOTA/val/images/P0003.png')[::-1]

plt.figure(figsize=(15,15))
plt.imshow(img)

In [ ]:
labels = open('/datasets/dataset/DOTA/val/labels/labelTxt-v1.0/P0003.txt', 'r').read()
labels = labels.split('\n')[2:-1]
labels = [l.split(' ') for l in labels]

In [ ]:
ann = labels[0]
ann

In [ ]:
box = np.int0(ann[:-2])
box = np.int0(list(zip(box[::2], box[1::2])))
box[:,0] = img.shape[0] - box[:,0]
box[:,1] = img.shape[1] - box[:,1]

print(box)

In [ ]:
draw = np.copy(img)
for ann in labels:
    box = np.int0(ann[:-2])
    box = np.int0(list(zip(box[::2], box[1::2])))
    box[:,0] = img.shape[0] - box[:,0]
    box[:,1] = img.shape[1] - box[:,1]
    
    cv2.drawContours(draw, [box], 0, (0,255,0), 2)

plt.figure(figsize=(15,15))
plt.imshow(draw)

In [ ]:
%cd /workdir/msc/RotationDetection/tools/gwd
!python test_hrsc2016.py --img_dir='/datasets/dataset/HRSC2016/HRSC2016/HRSC2016/Test/AllImages'  \
                         --gpu=1 \
                         --image_ext='.bmp' \
                         --test_annotation_path='/datasets/dataset/HRSC2016/HRSC2016/HRSC2016/Test/xmls'